In [1]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import csv
import re
import Levenshtein
from spellchecker import SpellChecker
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import json

C:\Users\Audi\Anaconda3\envs\dcd_llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Audi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
dfo = pd.read_csv("Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")
def standardize_tags(text):
    # Remove any double quotes around '{{' and '}}' and ensure the tags are uniform
    standardized_text = re.sub(r'[\'"]?\{\{([^}]*)\}\}[\'"]?', r'{{\1}}', text)
    return standardized_text

dfo['instruction'] = dfo['instruction'].apply(standardize_tags)
dfo['response'] = dfo['response'].apply(standardize_tags)

In [72]:
def expand_contractions(text, contractions):
    for contraction, expansion in contractions.items():
        expanded_form = expansion.split('/')[0].strip()  # Selecting the first (most common) expansion
        # Ensure the contraction is surrounded by word boundaries to prevent partial matches
        contraction_pattern = r'\b' + re.escape(contraction) + r'\b'
        text = re.sub(contraction_pattern, expanded_form, text, flags=re.IGNORECASE)
    return text

# Complete contractions dictionary
contractions = {
    "I'm": "I am",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",  # "he had" as the more common form
    "he'd've": "he would have",
    "he'll": "he shall",
    "he'll've": "he shall have",
    "he's": "he has",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has",
    "I'd": "I had",
    "I'd've": "I would have",
    "I'll": "I shall",
    "I'll've": "I shall have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it shall",
    "it'll've": "it shall have",
    "it's": "it has",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she shall",
    "she'll've": "she shall have",
    "she's": "she has",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that has",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there has",
    "they'd": "they had",
    "they'd've": "they would have",
    "they'll": "they shall",
    "they'll've": "they shall have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall",
    "what'll've": "what shall have",
    "what're": "what are",
    "what's": "what has",
    "what've": "what have",
    "when's": "when has",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has",
    "where've": "where have",
    "who'll": "who shall",
    "who'll've": "who shall have",
    "who's": "who has",
    "who've": "who have",
    "why's": "why has",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you shall",
    "you'll've": "you shall have",
    "you're": "you are",
    "you've": "you have"
}


# Applying the function to each row in the DataFrame
dfo['instruction'] = dfo['instruction'].apply(lambda x: expand_contractions(x, contractions))
dfo['response'] = dfo['response'].apply(lambda x: expand_contractions(x, contractions))

print(dfo['response'][2])

I can sense that you are seeking assistance with canceling your purchase with the purchase number {{Order Number}}. I apologize for any inconvenience caused, and I am here to guide you through the process.

To cancel your purchase, please follow these steps:

1. Log into your account: Visit our {{Online Company Portal Info}} and sign in using your credentials.
2. Locate your order: Once logged in, navigate to the {{Online Order Interaction}} or {{Online Order Interaction}} section to find the purchase with the order number {{Order Number}}.
3. Initiate the cancellation: Upon finding your purchase, look for the option labeled {{Online Order Interaction}} and click on it to initiate the cancellation process.
4. Confirm the cancellation: Our system might ask for confirmation or feedback regarding the cancellation. Kindly provide the necessary information to complete the process.
5. Seek additional support: If you encounter any difficulties or require further assistance, our dedicated cust

In [73]:
dfo.to_csv("spellcheck2.csv", sep=',', index=False, encoding='utf-8')

In [74]:
spell = SpellChecker()
def correct_typos(text):
    corrected_words = []
    # Split the text into words and other characters, keeping everything intact
    tokens = re.findall(r'\b\w+\b|[^\w]', text)
    
    for token in tokens:
        # Check if the token is a word that needs spellchecking, ignoring special tags
        if token.isalpha() and not re.match(r'\{\{.*\}\}', token):
            corrected_word = spell.correction(token)
            if corrected_word is not None:
                corrected_words.append(corrected_word)
            else:
                corrected_words.append(token)  # Append the original token if correction is None
        else:
            # Keep other tokens (like punctuation, special tags, etc.) as is
            corrected_words.append(token)

    # Reconstruct the text
    corrected_text = ''.join(corrected_words)
    return corrected_text

# Apply the function to the DataFrame after expanding contractions
dfo['instruction'] = dfo['instruction'].apply(correct_typos)
dfo['response'] = dfo['response'].apply(correct_typos)

In [2]:
#dfo.to_csv("spellcheck3.csv", sep=',', index=False, encoding='utf-8')
df = pd.read_csv("spellcheck3.csv")
dfo = pd.read_csv("Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")

In [27]:
def replace_tags_with_uid(text, tag_dict):
    """
    Replaces tags with unique identifiers and updates the tag dictionary.
    """
    tags = set(re.findall(r'\{\{(.*?)\}\}', text))
    for tag in tags:
        if tag not in tag_dict:
            tag_dict[tag] = f"UID{len(tag_dict) + 1}TAG"
        text = text.replace(f"{{{{{tag}}}}}", tag_dict[tag])
    return text

def revert_uid_to_tags(text, tag_dict):
    """
    Reverts unique identifiers back to original tags.
    """
    for tag, uid in tag_dict.items():
        text = text.replace(uid, f"{{{{{tag}}}}}")
    return text

def translate_text(text, translation_pipeline):
    """
    Translates the text using the provided translation pipeline.
    """
    return translation_pipeline(text)[0]['translation_text']

In [4]:
from transformers import pipeline

# Initialize the translation pipeline
translate_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-id", device=0, batch_size=16)

# DataFrame and tag dictionary
tag_dict = {}

# Translate 'instruction' and 'response' columns
for col in ['instruction']:
    df[f'{col}_translated'] = df[col].apply(lambda x: replace_tags_with_uid(x, tag_dict))
    df[f'{col}_translated'] = df[f'{col}_translated'].apply(lambda x: translate_text(x, translate_pipe))
    df[f'{col}_translated'] = df[f'{col}_translated'].apply(lambda x: revert_uid_to_tags(x, tag_dict))

C:\Users\Audi\Anaconda3\envs\dcd_llm\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [3]:
#df.to_csv("translated-ins.csv", sep=',', index=False, encoding='utf-8')
df = pd.read_csv("translated-ins.csv")
dfo = pd.read_csv("Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")

In [23]:
# Regular expression pattern to match text within {} or {{}}
pattern = r'\{\{?([^}]*)\}?\}'

# Set to store unique values
unique_values_og = set()
unique_values_tr = set()

# Iterate through each response and find matches
for response in df['instruction_translated']:
    matches = re.findall(pattern, response)
    unique_values_tr.update(matches)

    
# Iterate through each response and find matches
for response in df['instruction']:
    matches = re.findall(pattern, response)
    unique_values_og.update(matches)
    
# Print or process the unique values
print(unique_values_og)
print(unique_values_tr)

{'Order Number', 'Refund Amount', 'Person Name', 'Delivery City', 'Delivery Country', 'Invoice Number', 'Account Category', 'Currency Symbol', 'Account Type'}
{'Order Number', 'Refund Amount', 'Person Name', 'Delivery City', '\\cHffffff', 'Delivery Country', 'Invoice Number', 'Account Category', 'Currency Symbol', 'Account Type', '\\cH80FFFF'}


In [7]:
def process_text_for_translation(text, tag_dict):
    # Break text into sentences
    sentences = sent_tokenize(text)
    processed_sentences = []

    # Replace tags with unique placeholders
    for sentence in sentences:
        tags = re.findall(r'\{\{(.*?)\}\}', sentence)
        for tag in tags:
            if tag not in tag_dict:
                tag_dict[tag] = f"UID{len(tag_dict) + 1}TAG"
            sentence = sentence.replace(f"{{{{{tag}}}}}", tag_dict[tag])
        processed_sentences.append(sentence)

    return processed_sentences

def translate_and_reassemble(processed_sentences, translate_pipe, tag_dict):
    translated_sentences = []

    # Translate each sentence
    for sentence in processed_sentences:
        translated_sentence = translate_pipe(sentence, max_length=512, truncation=True)[0]['translation_text']
        translated_sentences.append(translated_sentence)

    # Reassemble text and restore original tags
    reassembled_text = ''
    for sentence in translated_sentences:
        for tag, placeholder in tag_dict.items():
            sentence = sentence.replace(placeholder, f"{{{{{tag}}}}}")
        reassembled_text += sentence + ' '

    return reassembled_text

def add_paragraph_spacing_to_lists(translated_text):
    """
    Adds paragraph spacing to numbered lists in the translated text.
    """
    # Regular expression to find the start of a numbered list
    start_of_list_pattern = r'([^\n])\n(1\.)'
    translated_text = re.sub(start_of_list_pattern, r'\1\n\n\2', translated_text)

    # Regular expression to find each number in the list after the first
    subsequent_numbers_pattern = r'(\d\..+?)(\n)(\d\.)'
    translated_text = re.sub(subsequent_numbers_pattern, r'\1\n\2\3', translated_text, flags=re.DOTALL)

    # Regular expression to find the end of a numbered list
    end_of_list_pattern = r'(\d\..+?)(\n)([^\n\d])'
    translated_text = re.sub(end_of_list_pattern, r'\1\n\n\2\3', translated_text, flags=re.DOTALL)

    return translated_text

# Initialize the translation pipeline
translate_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-id", device=0, batch_size=32)

# Sample text
sample_text = """
How splendid it is to embark on this exhilarating journey of modifying the data on your esteemed {{Account Category}} account!I am delighted to be your dedicated guide throughout this seamless process. I assure you that your account information will be accurately updated to reflect your evolving preferences and priorities. let us dive in together and bring a touch of personalization and excellence to your {{Account Category}} account. What specific details would you like to modify?out to us. We are here to help you.
"""

# Tag dictionary for placeholders
tag_dict = {}

# Process text for translation
processed_sentences = process_text_for_translation(sample_text, tag_dict)

# Translate and reassemble
translated_text = translate_and_reassemble(processed_sentences, translate_pipe, tag_dict)

# Add paragraph spacing around numbered lists
formatted_translated_text = add_paragraph_spacing_to_lists(translated_text)

print(formatted_translated_text)

Betapa senangnya memulai perjalanan yang menggembirakan ini untuk memodifikasi data pada akun {{Account Category}} Anda yang terhormat! Saya senang menjadi panduan khusus Anda sepanjang proses mulus ini. Saya yakinkan Anda bahwa informasi account Anda akan diperbarui secara akurat untuk mencerminkan preferensi dan prioritas berkembang Anda. Mari kita menyelam bersama-sama dan membawa sentuhan personalisasi dan keunggulan untuk account {{Account Category}} Anda. Rincian spesifik apa yang ingin kau ubah? Kami di sini untuk membantumu. 


In [7]:
# Function to process text for translation
def process_text_for_translation(text, tag_dict):
    sentences = sent_tokenize(text)
    processed_sentences = []
    for sentence in sentences:
        tags = re.findall(r'\{\{(.*?)\}\}', sentence)
        for tag in tags:
            if tag not in tag_dict:
                tag_dict[tag] = f"UNIQUE_TAG_{len(tag_dict)}"
            sentence = sentence.replace(f"{{{{{tag}}}}}", tag_dict[tag])
        processed_sentences.append(sentence)
    return processed_sentences

# Function to translate and reassemble text
def translate_and_reassemble(processed_sentences, translate_pipe, tag_dict):
    translated_sentences = []
    for sentence in processed_sentences:
        translated_sentence = translate_pipe(sentence, max_length=512, truncation=True)[0]['translation_text']
        translated_sentences.append(translated_sentence)

    reassembled_text = ''
    for sentence in translated_sentences:
        for tag, placeholder in tag_dict.items():
            sentence = sentence.replace(placeholder, f"{{{{{tag}}}}}")
        reassembled_text += sentence + ' '
    return reassembled_text

# Initialize the translation pipeline
translate_pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-id", device=0, batch_size=32)


# Tag dictionary for placeholders
tag_dict = {}

# Translate each response and store in a new column
df['response_translated'] = df['response'].apply(lambda x: translate_and_reassemble(process_text_for_translation(x, tag_dict), translate_pipe, tag_dict))

In [2]:
#df.to_csv("translatednotc.csv", sep=',', index=False, encoding='utf-8')
df = pd.read_csv("translatednotc.csv")

In [33]:
# Regular expression pattern to match text within {} or {{}}
pattern = r'\{\{?([^}]*)\}?\}'

# Set to store unique values
unique_values_og = set()
unique_values_tr = set()

# Iterate through each response and find matches
for response in df['instruction_translated_corrected']:
    matches = re.findall(pattern, response)
    unique_values_tr.update(matches)

    
# Iterate through each response and find matches
for response in df['instruction_translated']:
    matches = re.findall(pattern, response)
    unique_values_og.update(matches)
    
# Print or process the unique values
print(unique_values_og)
print(unique_values_tr)

{'Order Number', 'Refund Amount', 'Person Name', 'Delivery City', '\\cHffffff', 'Delivery Country', 'Invoice Number', 'Account Category', 'Currency Symbol', 'Account Type', '\\cH80FFFF'}
{'Order Number', 'Refund Amount', 'Person Name', 'Delivery City', 'Delivery Country', 'Invoice Number', 'Account Category', 'Currency Symbol', 'Account Type'}


In [31]:
df[df['instruction_translated'].str.contains(r'\\cHffffff|\\cH80FFFF')]

,flags,instruction,category,intent,response,instruction_translated,response_translated,response_translated_corrected,original_tag_count,corrected_tag_count,tag_count_match,instruction_translated_corrected
1968,BCLQWZ,wana swap a damn product of order {{Order Numb...,ORDER,change_order,Grateful you reached out to us for assistance ...,{\cHffffff}{{Order Number}} tolong aku.,Bersyukur kau membantu kami untuk menukar prod...,Bersyukur kau membantu kami untuk menukar prod...,1,1,True,{}{{Order Number}} tolong aku.
7269,BILZ,where to lodge a reclamation against your busi...,FEEDBACK,complaint,I genuinely empathize with your need to lodge ...,{\cH80FFFF}Di mana harus mengajukan reklamasi ...,Aku benar-benar berempati dengan kebutuhan And...,Aku benar-benar berempati dengan kebutuhan And...,0,0,True,{}Di mana harus mengajukan reklamasi terhadap ...
9424,BLQWZ,wana talk to a goddamn person,CONTACT,contact_human_agent,we are here to help! I take note that you are ...,{\cHffffff}Bicaralah dengan seseorang.,Kami di sini untuk membantu! Saya memperhatika...,Kami di sini untuk membantu! Saya memperhatika...,0,0,True,{}Bicaralah dengan seseorang.
16032,BILMZ,where to ask for reimbursement of money ?,REFUND,get_refund,I can see that you are looking for information...,{\cHffffff}Kemana harus minta ganti rugi uang?,Saya dapat melihat bahwa Anda sedang mencari i...,Saya dapat melihat bahwa Anda sedang mencari i...,3,3,True,{}Kemana harus minta ganti rugi uang?
16493,BILZ,how can I ask for a rebate ?,REFUND,get_refund,I can sense that you are interested in learnin...,{\cH80FFFF}Bagaimana aku bisa meminta potongan...,Aku bisa merasakan bahwa kau tertarik untuk be...,Aku bisa merasakan bahwa kau tertarik untuk be...,0,0,True,{}Bagaimana aku bisa meminta potongan harga?


In [4]:
# Regular expression pattern to match text within {} or {{}}
pattern = r'\{\{?([^}]*)\}?\}'

# Set to store unique values
unique_values = set()

# Iterate through each response and find matches
for response in df['response_translated']:
    matches = re.findall(pattern, response)
    unique_values.update(matches)

    
# Print or process the unique values
print(unique_values)

{'Company', 'Order Number', 'Cancellation Policy', 'Batal Order', 'Free Akun', 'Cancel Purchase', 'Customer Support Phone Number', 'Online Order Interaction', '{{Cancellation Policy', 'Online Company Portal Info', 'Customer Support Hours', 'Upgrade ke {{Cancellation Policy', 'Switch ke {{Cancellation Policy', 'Company Account', 'Website URL'}


In [5]:
# Regular expression pattern to match text within {} or {{}}
pattern = r'\{\{?([^}]*)\}?\}'

# Set to store unique values
unique_values = set()

# Iterate through each response and find matches
for response in df['response']:
    matches = re.findall(pattern, response)
    unique_values.update(matches)

    
# Print or process the unique values
print(unique_values)

{'Upgrade Account Type', 'Track Reimbursement', 'Month', 'Order Status', 'Profile Settings', 'User Account Settings', 'User Account Recovery', 'Compensation Type', 'Company Account', 'Company Phone Number', 'Login steps', 'Rebate ID', 'Customer Support Days', 'Product Feedback Email', 'Restore Password', 'Case Number', 'Change Account', 'Basic', 'Standard Shipping Days', 'Restore Access Key', 'PIN Reset Page URL', 'Choose the {{Account Type', 'Reset Password', 'Reset Key', 'Switch Plan', 'Password and Security', 'Delivery Country', 'Change Key', 'Feedback Email', 'Rebate Tracking Number', 'Feedback Email Address', 'Online Marketplace', 'Express Shipping Days', 'Non-receipt of Goods Refund Time', 'Online Customer Support Channel', 'Closing Time', 'Cancellation Policy', 'Social Media Platform', 'Forgot User Account Key', 'Retrieve PIN Code', 'Login Page URL', 'Account Management', 'Complaint Hotline Number', 'Third Step', 'Order Tracking', 'Live Chat', 'Account Details', 'PIN Code', 'Use

In [12]:
def replace_or_fallback_tags(original_text, translated_text):
    original_tags = re.findall(r'\{\{(.*?)\}\}', original_text)
    translated_tags = re.findall(r'\{\{(.*?)\}\}', translated_text)

    for i, orig_tag in enumerate(original_tags):
        trans_tag = translated_tags[i] if i < len(translated_tags) else orig_tag
        pattern = rf'\{{{{({re.escape(trans_tag)})\}}}}'
        translated_text = re.sub(pattern, f'{{{{{orig_tag}}}}}', translated_text, count=1)

    return translated_text

df['response_translated_corrected'] = df.apply(lambda row: replace_or_fallback_tags(row['response'], row['response_translated']), axis=1)

In [13]:
# Regular expression pattern to match text within {} or {{}}
pattern = r'\{\{?([^}]*)\}?\}'

# Set to store unique values
unique_values = set()

# Iterate through each response and find matches
for response in df['response_translated_corrected']:
    matches = re.findall(pattern, response)
    unique_values.update(matches)

    
# Print or process the unique values
print(unique_values)

{'Upgrade Account Type', 'Track Reimbursement', 'Month', 'Order Status', 'Profile Settings', 'User Account Settings', 'User Account Recovery', 'Compensation Type', 'Company Account', 'Company Phone Number', 'Login steps', 'Rebate ID', 'Customer Support Days', 'Product Feedback Email', 'Restore Password', 'Case Number', 'Change Account', 'Basic', 'Standard Shipping Days', 'Restore Access Key', 'PIN Reset Page URL', 'Choose the {{Account Type', 'Reset Password', 'Reset Key', 'Switch Plan', 'Password and Security', 'Delivery Country', 'Change Key', 'Feedback Email', 'Rebate Tracking Number', 'Feedback Email Address', 'Online Marketplace', 'Express Shipping Days', 'Non-receipt of Goods Refund Time', 'Online Customer Support Channel', 'Closing Time', 'Cancellation Policy', 'Social Media Platform', 'Forgot User Account Key', 'Retrieve PIN Code', 'Login Page URL', 'Account Management', 'Complaint Hotline Number', 'Third Step', 'Order Tracking', 'Live Chat', 'Account Details', 'PIN Code', 'Use

In [15]:
def remove_numbers_from_tags(text):
    # Remove numbers from double brace tags
    return re.sub(r'(\{\{[^}]*\})\d+', r'\1', text)

def count_double_brace_values(text):
    # Count the number of double brace values
    return len(re.findall(r'\{\{[^}]*\}\}', text))

# Apply removal of numbers and count before processing
df['original_tag_count'] = df['response'].apply(count_double_brace_values)

# Processing and correcting the translated text
df['response_translated_corrected'] = df.apply(lambda row: replace_or_fallback_tags(row['response'], row['response_translated']), axis=1)
df['response_translated_corrected'] = df['response_translated_corrected'].apply(remove_numbers_from_tags)

# Count after processing
df['corrected_tag_count'] = df['response_translated_corrected'].apply(count_double_brace_values)

# Verify if the counts match
df['tag_count_match'] = df['original_tag_count'] == df['corrected_tag_count']

# Check the results
print(df[['original_tag_count', 'corrected_tag_count', 'tag_count_match']])

       original_tag_count  corrected_tag_count  tag_count_match
0                       1                    1             True
1                       1                    1             True
2                       9                    9             True
3                       9                    9             True
4                       9                    9             True
...                   ...                  ...              ...
26867                   2                    2             True
26868                   1                    1             True
26869                   2                    2             True
26870                   0                    0             True
26871                   0                    0             True

[26872 rows x 3 columns]


In [19]:
df[df['tag_count_match'] == False]

,flags,instruction,category,intent,response,instruction_translated,response_translated,response_translated_corrected,original_tag_count,corrected_tag_count,tag_count_match
12,BL,problem with canceling purchase {{Order Number}},ORDER,cancel_order,I am aware that you are facing difficulties ca...,masalah dengan membatalkan {{Order Number}},Saya sadar bahwa Anda menghadapi kesulitan mem...,Saya sadar bahwa Anda menghadapi kesulitan mem...,3,2,False
17,BCL,"I bought the same item twice, cancel order {{O...",ORDER,cancel_order,I understand the frustration that arises when ...,"Aku membeli barang yang sama dua kali, membata...",Saya mengerti frustrasi yang muncul ketika mem...,Saya mengerti frustrasi yang muncul ketika mem...,9,8,False
40,BLQ,i need help to cancel purchase {{Order Number}},ORDER,cancel_order,Thank you for trusting to us for assistance wi...,i need bantuan untuk membatalkan pembelian {{O...,Terima kasih telah mempercayai kami untuk bant...,Terima kasih telah mempercayai kami untuk bant...,9,8,False
49,BL,assistance with canceling purchase {{Order Num...,ORDER,cancel_order,I have grasped that you require assistance wit...,bantuan dengan membatalkan pembelian {{Order N...,Saya telah memahami bahwa Anda membutuhkan ban...,Saya telah memahami bahwa Anda membutuhkan ban...,9,8,False
55,BL,assistance with canceling purchase {{Order Num...,ORDER,cancel_order,I am clearly cognizant your need for assistanc...,bantuan dengan membatalkan pembelian {{Order N...,Saya jelas mengakui kebutuhan Anda untuk bantu...,Saya jelas mengakui kebutuhan Anda untuk bantu...,10,9,False
...,...,...,...,...,...,...,...,...,...,...,...
25800,BIL,what is order {{Order Number}} current status?,ORDER,track_order,Your message means a lot! I am aligned with th...,apa status {{Order Number}} saat ini?,Pesan Anda berarti banyak! Saya selaras dengan...,Pesan Anda berarti banyak! Saya selaras dengan...,3,2,False
25865,BL,see the ETA of the purchase {{Order Number}},ORDER,track_order,I hear you! I am following you that you would ...,lihat ETA pembelian {{Order Number}},Aku mendengarmu! Saya mengikuti Anda bahwa And...,Aku mendengarmu! Saya mengikuti Anda bahwa And...,3,2,False
26032,BCIL,"I have to see the status of my rebate, how to ...",REFUND,track_refund,Thank you for raising your concern about check...,"Saya harus melihat status rebate saya, bagaima...",Terima kasih untuk meningkatkan kekhawatiran A...,Terima kasih untuk meningkatkan kekhawatiran A...,3,2,False
26201,BILM,where do I check if there are any news on the ...,REFUND,track_refund,We understand your eagerness to stay updated o...,di mana saya memeriksa apakah ada berita pada ...,Kami memahami keinginan Anda untuk tetap diper...,Kami memahami keinginan Anda untuk tetap diper...,3,2,False


In [32]:
# Function to replace specific mistranslations
def correct_mistranslations(text):
    corrections = {
        r'\bi\b': 'saya',
        r'\bneed\b': 'membutuhkan',
        r'\bdont\b': 'tidak',
        r"\bdon't\b": 'tidak',
        r'\biwant\b': 'saya ingin'
    }
    for wrong, correct in corrections.items():
        text = re.sub(wrong, correct, text, flags=re.IGNORECASE)
    return text

def clean_tags(translated_text):
    # Remove unwanted characters
    translated_text = translated_text.replace('\\cHffffff', '').replace('\\cH80FFFF', '').replace('{}', '')

    return translated_text

# Apply corrections to the DataFrame
df['instruction_translated'] = df['instruction_translated'].apply(correct_mistranslations)
df['instruction_translated_corrected'] = df['instruction_translated'].apply(clean_tags)

In [14]:
df.to_csv("translated-ins1.csv", sep=',', index=False, encoding='utf-8')

In [35]:
# Select specific columns and rename as needed
new_df = df[['flags', 'category', 'intent', 'instruction_translated_corrected', 'response_translated_corrected']].copy()
new_df.rename(columns={'instruction_translated_corrected': 'instruction', 'response_translated_corrected': 'response'}, inplace=True)

# Save to CSV
new_df.to_csv('translated_final.csv', index=False)

In [36]:
df = pd.read_csv("translated_final.csv")

In [38]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Capitalize the first letter of each sentence
    text = re.sub('([.!?]\s*)([a-z])', lambda p: p.group(1) + p.group(2).upper(), text)

    # Capitalize the very first letter if it's lowercase
    text = text[0].upper() + text[1:] if text else text

    return text

df['instruction'] = df['instruction'].apply(preprocess_text)
df['response'] = df['response'].apply(preprocess_text)

In [20]:
original_placeholders = {}

pattern = r'\{\{(.*?)\}\}'

with open('Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv', 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        matches = re.findall(pattern, row['response'])
        for match in matches:
            original_placeholders[match] = match  # key and value are the same

# Print to check
print(original_placeholders)

{'Order Number': 'Order Number', 'Online Company Portal Info': 'Online Company Portal Info', 'Online Order Interaction': 'Online Order Interaction', 'Customer Support Hours': 'Customer Support Hours', 'Customer Support Phone Number': 'Customer Support Phone Number', 'Website URL': 'Website URL', 'Cancel Purchase': 'Cancel Purchase', 'Company': 'Company', 'Company Account': 'Company Account', 'Cancellation Policy': 'Cancellation Policy', 'Company Name': 'Company Name', 'Refund Policy': 'Refund Policy', 'Return Policy': 'Return Policy', 'Confirm Cancellation': 'Confirm Cancellation', 'My Purchases': 'My Purchases', 'Customer Support Email': 'Customer Support Email', 'Customer Support Team': 'Customer Support Team', 'Remove': 'Remove', 'Invoice Number': 'Invoice Number', 'Person Name': 'Person Name', 'Salutation': 'Salutation', 'Client Last Name': 'Client Last Name', 'Date Range': 'Date Range', 'Client First Name': 'Client First Name', 'Date of the Invoice': 'Date of the Invoice', 'Date':

In [4]:
# Path to your CSV file
csv_file_path = 'translated_final.csv'
# Path for the output JSON file
json_file_path = 'bitext.json'

# Function to convert CSV to JSON
def csv_to_json(csv_file_path, json_file_path):
    json_array = []

    # Read the CSV and add the data to a dictionary
    with open(csv_file_path, encoding='utf-8') as csvf:
        csv_reader = csv.DictReader(csvf)
        for row in csv_reader:
            json_entry = {
                'instruction': row['instruction'],
                'input': row['response'],  # Assuming 'response' in CSV is the 'input' in JSON
                'response': row['response']
            }
            json_array.append(json_entry)

    # Write the dictionary to a JSON file
    with open(json_file_path, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(json_array, indent=4))

# Convert CSV to JSON
csv_to_json(csv_file_path, json_file_path)

In [2]:
df = pd.read_csv("translated_final.csv")
def remove_numbers_from_tags_v2(text):
    # Regular expression to find double brace tags with or without numbers at the end
    tag_pattern = r'\{\{[^}]*\}\}\d*'

    # Function to process each match
    def process_match(match):
        # Remove trailing digits from the match
        return re.sub(r'\d+$', '', match.group())

    # Replace the tag with number using process_match function
    return re.sub(tag_pattern, process_match, text)

# Apply the function to the 'response' column
df['response'] = df['response'].apply(remove_numbers_from_tags_v2)
df.to_csv("translated_final1.csv", sep=',', index=False, encoding='utf-8')

In [4]:
df['category'].nunique()
df['intent'].nunique()

27

In [2]:
df = pd.read_csv("translated_final1.csv") 
# Process the dataframe to fit the gpteacher format
df['input'] = df[['flags', 'intent', 'category']].agg(', '.join, axis=1)
df1 = df[['instruction', 'input', 'response']]
df1.to_parquet('bitext.parquet', index=False)

In [3]:
df1.head()

,instruction,input,response
0,pertanyaan tentang membatalkan urutan {{Order ...,"B, cancel_order, ORDER",Saya mengerti Anda memiliki pertanyaan mengena...
1,saya memiliki pertanyaan tentang membatalkan u...,"BQZ, cancel_order, ORDER",Saya telah diberitahu bahwa Anda memiliki pert...
2,saya membutuhkan help membatalkan pembelian {{...,"BLQZ, cancel_order, ORDER",Saya dapat merasakan bahwa Anda mencari bantua...
3,Saya perlu membatalkan {{Order Number}},"BL, cancel_order, ORDER",Saya mengerti bahwa Anda membutuhkan bantuan d...
4,"Saya tidak mampu memesan ini, membatalkan pemb...","BCELN, cancel_order, ORDER",Saya peka terhadap fakta bahwa Anda menghadapi...


In [10]:
df1r = df1.sample(frac=0.1, random_state=42)
df1r.to_parquet('bitext_0.1.parquet', index=False)

In [7]:
df = pd.read_csv("translated_final1.csv")
df['intent'].value_counts()

edit_account                1000
switch_account              1000
check_invoice               1000
complaint                   1000
contact_customer_service    1000
delivery_period              999
registration_problems        999
check_payment_methods        999
contact_human_agent          999
payment_issue                999
newsletter_subscription      999
get_invoice                  999
place_order                  998
cancel_order                 998
track_refund                 998
change_order                 997
get_refund                   997
create_account               997
check_refund_policy          997
review                       997
set_up_shipping_address      997
delivery_options             995
delete_account               995
recover_password             995
track_order                  995
change_shipping_address      973
check_cancellation_fee       950
Name: intent, dtype: int64

In [8]:
df['category'].value_counts()

ACCOUNT         5986
ORDER           3988
REFUND          2992
INVOICE         1999
CONTACT         1999
PAYMENT         1998
FEEDBACK        1997
DELIVERY        1994
SHIPPING        1970
SUBSCRIPTION     999
CANCEL           950
Name: category, dtype: int64